In [1]:
import sys

if ".." not in sys.path:
    sys.path.append("..")

In [2]:
import pickle 

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import networkx as nx
import cobra

from pprint import pp
from molmass import Formula
from cobra.io import read_sbml_model
from experiments.fast_dFBA import ConstantBounds
from utils.cobra_utils import get_or_create_exchange
from parameters.drawdown import *
from plotting.diagnostic import uptakes_and_secretions, plot_pathway, plot_metabolite_fluxes


MODEL_FILE = "../model/Rpom_05.xml"
BIOMASS_ID = "Rpom_hwa_biomass"
DATA_FILE = "../data/clean/CUE2/dFBA.pkl"


# Load and set up model
model = read_sbml_model(MODEL_FILE)

# Set maintenance to 25 mmol/gDW/h
atpm = model.reactions.get_by_id("ATPM")
atpm.bounds = (25, 25)

ex_glc = model.reactions.get_by_id("EX_glc")
ex_ace = get_or_create_exchange(model, "ACET[e]")

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [21]:
df = pd.read_excel("../data/raw/growth/essentiality - July 2023.xlsx")
df = df[~df["Mutant (yes/no)"].isna()]
df["Essential"] = (df["Mutant (yes/no)"] == "no")

In [23]:
# Get fluxes when running on glucose, acetate
with model:
    ex_glc.bounds = (-10, 0)
    sol_glc = model.optimize()
    mu_glc = sol_glc.objective_value
    model.summary()
    print(f"Glucose growth: {sol_glc.objective_value:.2f}")

Glucose growth: 0.72


In [49]:
model.reactions.get_by_id("ACETATE--COA-LIGASE-RXN").genes

frozenset({<Gene G1G48-1850 at 0x7f8441e4cdc0>,
           <Gene G1G48-1997 at 0x7f8441e4cd90>,
           <Gene G1G48-2841 at 0x7f8441e4cd60>,
           <Gene G1G48-4366 at 0x7f8441e4cdf0>})

In [46]:
import collections

collections.Counter([gene.id.startswith("G1G48") for gene in model.genes])
# collections.Counter([gene.name == gene.id for gene in model.genes])

Counter({True: 1207, False: 43})